In [4]:
# Pipeline de processamento simplificado dos dados existentes
import subprocess
import sys
import os
import pandas as pd
from datetime import datetime
from pathlib import Path

def check_data_exists():
    """Verifica se os dados brutos existem"""
    print("📊 VERIFICANDO DADOS EXISTENTES")
    print("=" * 40)
    
    data_paths = {
        "Open-Meteo Historical Forecast": "../data/raw/Open-Meteo Historical Forecast/",
        "Open-Meteo Historical Weather": "../data/raw/Open-Meteo Historical Weather/", 
        "INMET": "../data/raw/INMET/"
    }
    
    data_summary = {}
    for name, path in data_paths.items():
        if Path(path).exists():
            # Contar todos os arquivos (CSV e JSON)
            csv_files = list(Path(path).glob("*.csv"))
            json_files = list(Path(path).glob("*.json"))
            total_files = len(csv_files) + len(json_files)
            
            data_summary[name] = {
                "exists": True,
                "files": total_files,
                "csv_files": len(csv_files),
                "json_files": len(json_files),
                "path": path
            }
            print(f"✅ {name}: {total_files} arquivo(s) ({len(csv_files)} CSV, {len(json_files)} JSON)")
        else:
            data_summary[name] = {
                "exists": False,
                "files": 0,
                "path": path
            }
            print(f"❌ {name}: Não encontrado")
    
    return data_summary

def run_processing_script():
    """Executa o script de processamento personalizado"""
    print("\n🔄 Executando processamento dos dados existentes")
    print("=" * 50)
    
    script_path = Path("../scripts/process_existing_data.py")
    
    try:
        result = subprocess.run([sys.executable, str(script_path)], 
                              capture_output=True, text=True, timeout=600)
        
        if result.returncode == 0:
            print("✅ Processamento - SUCESSO")
            if result.stdout:
                print("📤 Output:")
                print(result.stdout)
        else:
            print("❌ Processamento - ERRO")
            print("💥 Error output:")
            print(result.stderr)
            
        return result.returncode == 0
            
    except subprocess.TimeoutExpired:
        print("⏰ Processamento - TIMEOUT (>10min)")
        return False
    except FileNotFoundError:
        print(f"📁 Script não encontrado: {script_path}")
        return False
    except Exception as e:
        print(f"💥 Erro executando processamento: {e}")
        return False

def check_processed_data():
    """Verifica se os dados foram processados com sucesso"""
    print("\n📋 VERIFICANDO DADOS PROCESSADOS")
    print("=" * 40)
    
    expected_files = [
        "../data/processed/openmeteo_historical_forecast.csv",
        "../data/processed/openmeteo_historical_weather.csv", 
        "../data/processed/dados_inmet_processados.csv",
        "../data/processed/atmospheric_features_149vars.csv",
        "../data/processed/surface_features_25vars.csv"
    ]
    
    processed_summary = {}
    
    for file_path in expected_files:
        path = Path(file_path)
        if path.exists():
            try:
                df = pd.read_csv(path)
                processed_summary[path.name] = {
                    "exists": True,
                    "shape": df.shape,
                    "size_mb": round(path.stat().st_size / 1024 / 1024, 2)
                }
                print(f"✅ {path.name}: {df.shape} - {processed_summary[path.name]['size_mb']}MB")
            except Exception as e:
                processed_summary[path.name] = {
                    "exists": True,
                    "error": str(e)
                }
                print(f"⚠️ {path.name}: Erro ao ler - {e}")
        else:
            processed_summary[path.name] = {"exists": False}
            print(f"❌ {path.name}: Não encontrado")
    
    return processed_summary

print("🚀 INICIANDO PIPELINE DE PROCESSAMENTO DE DADOS")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# 1. Verificar dados brutos
data_summary = check_data_exists()
data_available = any(ds["exists"] for ds in data_summary.values())

if data_available:
    print(f"\n✅ Dados brutos encontrados!")
    
    # 2. Executar processamento
    processing_success = run_processing_script()
    
    if processing_success:
        # 3. Verificar resultados
        processed_summary = check_processed_data()
        
        # 4. Relatório final
        processed_files = sum(1 for ps in processed_summary.values() if ps.get("exists", False))
        
        print(f"\n🎯 PIPELINE CONCLUÍDO!")
        print("=" * 30)
        print(f"✅ {processed_files}/{len(processed_summary)} arquivos processados")
        print(f"📊 Dados prontos para análise e treinamento")
        
        if processed_files >= 2:  # Pelo menos dados básicos
            print("\n🔗 Próximos passos:")
            print("   1. ✅ Execute: notebooks/1_exploratory_analysis.ipynb")
            print("   2. ✅ Execute: notebooks/2_model_training.ipynb") 
            print("   3. ✅ Execute: notebooks/3_model_evaluation.ipynb")
        else:
            print("\n⚠️ Dados insuficientes para treinamento completo")
            print("   Mas você pode executar a análise exploratória")
    else:
        print("\n❌ Falha no processamento dos dados")
        print("📋 Verifique os logs acima para detalhes")
        
else:
    print("\n❌ Dados brutos não encontrados!")
    print("📁 Certifique-se de que existem dados em:")
    for name, ds in data_summary.items():
        if not ds["exists"]:
            print(f"   - {ds['path']}")

print(f"\n📝 Log completo salvo em: logs/")
print(f"📊 Para análise manual, verifique: data/processed/")


🚀 INICIANDO PIPELINE DE PROCESSAMENTO DE DADOS
📅 2025-06-06 14:58:45
📊 VERIFICANDO DADOS EXISTENTES
✅ Open-Meteo Historical Forecast: 14 arquivo(s) (0 CSV, 14 JSON)
✅ Open-Meteo Historical Weather: 75 arquivo(s) (50 CSV, 25 JSON)
✅ INMET: 0 arquivo(s) (0 CSV, 0 JSON)

✅ Dados brutos encontrados!

🔄 Executando processamento dos dados existentes
✅ Processamento - SUCESSO
📤 Output:
🚀 INICIANDO PROCESSAMENTO DOS DADOS EXISTENTES
📅 2025-06-06 14:58:46
✅ Diretórios configurados

🔄 Processando Open-Meteo Historical Forecast...
📁 Encontrados 14 arquivos JSON
✅ Carregado: chunk_20221231_to_20230331.json - (2184, 49)
✅ Carregado: chunk_20230930_to_20231229.json - (2184, 49)
✅ Carregado: chunk_20240330_to_20240628.json - (2184, 49)
✅ Carregado: chunk_20221001_to_20221230.json - (2184, 49)
✅ Carregado: chunk_20220101_to_20220401.json - (2184, 49)
✅ Salvo: /Users/gabrielschuler/Developer/Fiap/Challenge/data/processed/openmeteo_historical_forecast.csv
📊 Shape final: (10920, 49)
🗓️ Período: 2022-01-0